In [ ]:
from enhancedAgent_v2 import *
from pythonTask import *

In [2]:
调用智能体的原则='''

### 智能体协同七诫
1. **能力透明**  
   *每个智能体必须显式声明能力边界*  
   > 教训：未验证方法存在导致流程崩溃

2. **状态显化**  
   *跨智能体数据必须通过受控总线传递*  
   > 教训：隐式变量共享引发不可测错误

3. **错误熔断**  
   *单点故障必须隔离在发生层*  
   > 教训：模型错误扩散至文件验证层

4. **验证前移**  
   *关键检查点嵌入执行流中部*  
   > 教训：后置验证增加回溯成本

5. **知识反哺**  
   *异常模式须转化为防范知识*  
   > 教训：相同错误在不同环节重复发生

6. **熵感控制**  
   *系统需持续监测并压缩认知混乱度*  
   > 教训：错误处理逻辑自身产生新熵

7. **量子信任**  
   *所有交互需携带可验证执行证明*  
   > 教训：依赖未经验证的输出导致连环失败

### 终极法则
**智能体不是工具而是生态**  
> 成功系统 = 清晰能力边界 × 严格状态管理 × 自愈机制  
> 失败系统 = 模糊责任 + 隐式依赖 + 脆弱传递  

这七条戒律构成智能体协同的免疫基因，使系统从机械执行进化为有机生命体。 
'''

# 销售数据分析

In [3]:
# 销售分析任务
# 设置代理服务器
import os
# os.environ['http_proxy'] = 'http://127.0.0.1:7890'
# os.environ['https_proxy'] = 'http://127.0.0.1:7890'

# os.environ["AGENT_MAX_TOKENS"] = "1000000"
from pythonTask import *
from knowledge_agent import promgraming_knowledge

 
# llm=llm_claude_sonnet_4
# llm=llm_gemini_2_5_pro_preview_06_05_google
# llm=llm_deepseek
llm=llm_deepseek

# 实例化 MultiStepAgent_v2 时不传入 agent_specs
multi_agent = MultiStepAgent_v2(llm=llm_deepseek)
# multi_agent.loadKnowledge(调用智能体的原则)

# 使用 register_agent 动态注册 Agent
general_agent = Agent(llm=llm_deepseek)
general_agent.api_specification='''
general_agent,擅长执行各种任务
'''
multi_agent.register_agent(
    name="general_agent",
    instance=general_agent
)

verify_agent = Agent(llm=llm_deepseek)
verify_agent.loadKnowledge(promgraming_knowledge)
verify_agent.loadKnowledge('''
如果指令要求你验证任务，你应该调用调用llm_gemini_2_flash_openrouter语言模型验证任务
''')
verify_agent.api_specification='''
verify_agent,擅长验证任务
'''
multi_agent.register_agent(
    name="verify_agent",
    instance=verify_agent
)

document_agent = Agent(llm=llm_deepseek)
document_agent.loadKnowledge(promgraming_knowledge)
document_agent.loadKnowledge('如果指令要求你写文档，你应该调用llm_gemini_2_flash_openrouter语言模型生成文档')
document_agent.api_specification='''
文档Agent,擅长调用gemini模型写文档
'''
multi_agent.register_agent(
    name="document_agent",
    instance=document_agent
)


# 示例主指令
# main_instruction = "请用python写一个hello world程序"
main_instruction = """

# 销售数据分析任务

sales_data.csv是销售数据文件，请使用此文件进行数据分析。

# 规则
1. 不要生成图表
2. 报告中必须包含每个地区，每个产品，每个销售人员的销售额
3. 分析报告保存到sales_analysis_report.md
4. 分析报告必须调用gemini模型生成

"""

# 执行多步骤任务
result = multi_agent.execute_multi_step(main_instruction)
print("多步骤执行结果：")
print(result)

2025-06-18 03:16:04,170 - enhancedAgent_v2 - DEBUG - 已注册 Agent: general_agent
2025-06-18 03:16:04,174 - enhancedAgent_v2 - DEBUG - 已注册 Agent: verify_agent
2025-06-18 03:16:04,177 - enhancedAgent_v2 - DEBUG - 已注册 Agent: document_agent
2025-06-18 03:16:04,207 - enhancedAgent_v2 - DEBUG - 从对象中解析到步骤，共 6 个步骤
2025-06-18 03:16:04,209 - enhancedAgent_v2 - DEBUG - 生成计划: [{'id': 'info1', 'name': '收集销售数据文件信息', 'instruction': '确认sales_data.csv文件的存在，并获取其路径。', 'agent_name': 'general_agent', 'instruction_type': 'information', 'phase': 'information', 'expected_output': 'sales_data.csv文件的路径确认。', 'prerequisites': '无', 'status': 'pending'}, {'id': 'info2', 'name': '收集分析报告要求', 'instruction': '确认分析报告的要求，包括不生成图表，报告中必须包含每个地区、每个产品、每个销售人员的销售额，分析报告保存到sales_analysis_report.md，且必须调用gemini模型生成。', 'agent_name': 'general_agent', 'instruction_type': 'information', 'phase': 'information', 'expected_output': '明确的分析报告要求列表。', 'prerequisites': '无', 'status': 'pending'}, {'id': 'exec1', 'name': '读取销售数据', 'instruction': '使用


当前执行计划:
[
  {
    "id": "info1",
    "name": "收集销售数据文件信息",
    "instruction": "确认sales_data.csv文件的存在，并获取其路径。",
    "agent_name": "general_agent",
    "instruction_type": "information",
    "phase": "information",
    "expected_output": "sales_data.csv文件的路径确认。",
    "prerequisites": "无",
    "status": "pending"
  },
  {
    "id": "info2",
    "name": "收集分析报告要求",
    "instruction": "确认分析报告的要求，包括不生成图表，报告中必须包含每个地区、每个产品、每个销售人员的销售额，分析报告保存到sales_analysis_report.md，且必须调用gemini模型生成。",
    "agent_name": "general_agent",
    "instruction_type": "information",
    "phase": "information",
    "expected_output": "明确的分析报告要求列表。",
    "prerequisites": "无",
    "status": "pending"
  },
  {
    "id": "exec1",
    "name": "读取销售数据",
    "instruction": "使用pandas读取sales_data.csv文件中的数据。",
    "agent_name": "general_agent",
    "instruction_type": "execution",
    "phase": "execution",
    "expected_output": "sales_data.csv文件中的数据被成功读取并存储在DataFrame中。",
    "prerequisites": "sales_data.csv文件路径确认。",
    "status"

2025-06-18 03:16:04,541 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


我可以帮你确认文件是否存在并获取路径。在Python中，我们可以使用`os`模块来完成这个任务。以下是操作步骤：

1. 首先需要导入`os`模块
2. 然后使用`os.path.exists()`函数检查文件是否存在
3. 如果文件存在，可以使用`os.path.abspath()`获取完整路径

示例代码：
```python
import os

file_name = "sales_data.csv"
if os.path.exists(file_name):
    file_path = os.path.abspath(file_name)
    print(f"文件存在，路径是: {file_path}")
else:
    print("sales_data.csv文件不存在于当前目录")
```

你需要运行这段代码来确认文件是否存在。如果你需要，我可以帮你解释代码的任何部分，或者根据你的具体情况调整代码。

2025-06-18 03:16:14,736 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-06-18 03:16:21,814 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


...

{
  "valid": true,
  "missing_sections": [],
  "format_issues": [],
  "suggestions": [
    "报告内容被截断，请提供完整报告以便进行更全面的分析。",
    "可以考虑增加时间序列分析，观察销售额随时间的变化趋势。",
    "可以考虑增加客户细分分析，了解不同客户群体的购买行为。",
    "可以考虑增加产品组合分析，了解不同产品之间的关联性。",
    "可以考虑增加渠道分析，了解不同销售渠道的贡献。",
    "在业务建议部分，可以针对每个维度提供更具体、可操作的建议。"
  ]
}
任务完成：销售分析报告已通过验证
Thinker execute_stream
当前命令：# 执行任务

## 指令
检查sales_analysis_report.md文件是否存在，并确认报告内容是否符合要求。

## 前序步骤结果
步骤 收集销售数据文件信息 的结果:
我可以帮你确认文件是否存在并获取路径。在Python中，我们可以使用`os`模块来完成这个任务。以下是操作步骤：

1. 首先需要导入`os`模块
2. 然后使用`os.path.exists()`函数检查文件是否存在
3. 如果文件存在，可以使用`os.path.abspath()`获取完整路径

示例代码：
```python
import os

file_name = "sales_data.csv"
if os.path.exists(file_name):
    file_path = os.path.abspath(file_name)
    print(f"文件存在，路径是: {file_path}")
else:
    print("sales_data.csv文件不存在于当前目录")
```

你需要运行这段代码来确认文件是否存在。如果你需要，我可以帮你解释代码的任何部分，或者根据你的具体情况调整代码。
步骤 收集分析报告要求 的结果:
我理解你需要创建一个销售分析报告，具体要求如下：

1. **报告内容要求**：
   - 不生成图表（纯文本报告）
   - 包含每个地区的销售额分析
   - 包含每个产品的销售额分析
   - 包含每个销售人员的销售额分析

2. **输出要求**

2025-06-18 03:21:10,105 - root - INFO - === 开始评估 ===
2025-06-18 03:21:10,107 - root - INFO - 使用 1 个评估器进行评估...
2025-06-18 03:21:10,109 - root - INFO - 执行评估器 #1:
2025-06-18 03:21:10,305 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-06-18 03:21:14,934 - root - INFO - 评估器 #1 评估结果: 成功
2025-06-18 03:21:14,936 - root - INFO - === 评估总结 ===
2025-06-18 03:21:14,938 - root - INFO - 整体评估结果: 成功
2025-06-18 03:21:14,950 - root - INFO - 详细信息: 所有评估器都通过
#1: 程序成功执行且输出了预期结果，无错误信息，且包含断言验证且通过了验证
2025-06-18 03:21:14,953 - pythonTask - INFO - 开始生成指令最终结果
2025-06-18 03:21:14,956 - pythonTask - INFO - result.success: True
2025-06-18 03:21:14,960 - pythonTask - INFO - result.code: try:
    import os
    import json
    from langchain_openai import ChatOpenAI
    from dotenv import load_dotenv
    from autogen.code_utils import extract_code
    
    # 检查文件是否存在
    report_file = "sales_analysis_report.md"
    if not os.path.exists(report_file):
        raise Fi

# 销售分析报告验证结果

## 验证状态
✅ 报告验证通过 - `sales_analysis_report.md` 文件存在且内容符合要求

## 详细验证结果
1. **文件存在性检查**：成功找到报告文件
2. **内容完整性检查**：
   - 包含所有要求的分析维度（地区、产品、销售人员）
   - 使用专业的商业分析语言
   - 符合纯文本markdown格式要求

## 改进建议
虽然报告已符合基本要求，但以下优化建议可供参考：
1. 增加时间序列分析，观察销售额随时间变化趋势
2. 补充客户细分分析，了解不同客户群体购买行为
3. 添加产品组合分析，研究产品间关联性
4. 在业务建议部分提供更具体、可操作的建议

## 后续步骤
您现在可以：
1. 直接查看 `sales_analysis_report.md` 文件获取完整分析
2. 根据业务需求选择实施上述改进建议
3. 基于报告结论制定销售策略

需要我帮助实施任何改进建议或进行其他分析吗？

2025-06-18 03:21:29,667 - httpx - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"



决策结果: complete
原因: 所有步骤已成功完成，包括最后的验证步骤。销售分析报告已按要求生成并通过验证，原始目标已达成。
多步骤执行结果：

## 执行摘要
- 总步骤数: 6
- 已完成: 6
- 失败: 0
- 未执行: 0


继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
继续执行下一个步骤。
决策为完成执行。



In [ ]:
response=multi_agent.chat_stream('写个使用智能体的简短的经验教训总结')
for chunk in response:
    print(chunk, end='', flush=True)

# 计算器

In [ ]:
# 示例2：翻译模式（方案2）
llm = llm_deepseek

# 创建翻译模式的多步骤智能体（方案2：支持控制流）
workflow_agent = MultiStepAgent_v2(llm=llm, use_autonomous_planning=False)

# 注册智能体
coder_agent = Agent(llm=llm)
coder_agent.api_specification = '''
通用编程智能体,擅长编写python代码
'''
workflow_agent.register_agent(
    name="coder",
    instance=coder_agent
)

test_agent = Agent(llm=llm)
test_agent.api_specification = '''
软件测试智能体,擅长运行和测试python代码
'''
workflow_agent.register_agent(
    name="tester",
    instance=test_agent
)

# 翻译模式 - 用户提供详细步骤，AI翻译为JSON计划并执行
print("=== 翻译模式示例（方案2：动态控制流）===")
result = workflow_agent.execute_multi_step("""
1. coder: 实现一个简单的计算器类，包含加减乘除功能和完整单元测试
2. coder: 把代码保存到simple_calculator.py
3. tester: 运行simple_calculator.py，执行所有测试
4. decision_maker: 分析测试结果，测试通过则完成工作流，测试失败则状态转移至步骤3重新执行测试
""")

In [ ]:
if __name__ == "__main__":
    llm=llm_deepseek
    code_file_editor = Agent(llm=llm)
    code_file_editor.loadKnowledge('''
    ## 编程任务示例代码
    import aider_demo.aider_programming_demo
    instruction = f"保存代码到{file_name}\n#代码\n{code}" 
    edit_file_names=[{file_name}] # 要编辑的文件列表
    read_only_files=[] # 只读文件列表，只读文件不会被修改，是要编辑的文件依赖的文件
    result=aider_demo.aider_programming_demo.programming(instruction,edit_file_names,read_only_files) # 执行编程任务
    print(result) # 打印编程任务结果

    ## 如果指令是编写或修改或保存python文件，优先使用aider_demo.aider_programming_demo.programming函数执行编程任务修改python文件。如果programming函数失败，直接使用python代码修改文件。
                                ''')
    
    code_file_editor.api_specification="""
    python文件编辑Agent，擅长编辑python文件。
    
    输入：
    1. 代码或者写个代码的指令
    2. 要编辑的文件列表
    3. 只读文件列表，只读文件不会被修改，是要编辑的文件依赖的文件
    
    输出：
    1. 编辑后的代码
    2. 编辑后的代码说明
    
    # 示例
    #假设代码保存在变量code中
    response=code_file_editor.execute_stream(f'''
    把代码保存到/home/guci/myModule/AiResearch/math1.py

    # 代码
    {code}
                                            ''')
    result=None
    for chunk in response:
        result=chunk
        print(chunk,end='',flush=True)
        
    print(result.return_value)
    """
    
    code=''
    instruction=f'''
    请把代码保存到/home/guci/myModule/AiResearch/message_compress.py
    # 代码
    {code}
    '''
    response=code_file_editor.execute_stream(instruction)
    result=None
    for chunk in response:
        result=chunk
        print(chunk,end='',flush=True)
        
# ====== 测试和示例代码 ======

if __name__ == "__main__":
    # 测试代码：DeepSeek Prover 模型
    from langchain_openai import ChatOpenAI
    import os
    from dotenv import load_dotenv
    load_dotenv()
    
    llm_prover_openrouter = ChatOpenAI(
        temperature=0,
        model="deepseek/deepseek-prover-v2", 
        base_url='https://openrouter.ai/api/v1',
        api_key=os.getenv('OPENROUTER_API_KEY'),
    )

    x=llm_prover_openrouter.invoke("扩散模型架构是图灵完备的吗？")
    print(x.content)
    
    x=llm_prover_openrouter.invoke("请证明1+1=2，只允许使用纯逻辑")
    print(x.content)
    
    x=llm_prover_openrouter.invoke("图灵完备意味着能实现任意智能吗？")
    print(x.content)

    x=llm_prover_openrouter.invoke("请用transformer架构实现一个通用图灵机，它能够模拟任意图灵机。代码注释请用中文。")
    print(x.content)

    from pythonTask import llm_deepseek_r1
    x=llm_deepseek_r1.invoke("请用transformer架构实现一个通用图灵机，它能够模拟任意图灵机。单元测试中模拟一个简单的图灵机。代码注释请用中文。")
    print(x.content)

if __name__ == "__main__":
    llm=llm_deepseek
    coder = Agent(llm=llm)
    code_file_editor = Agent(llm=llm)
    code_file_editor.loadKnowledge('''
    ## 编程任务示例代码
    import aider_demo.aider_programming_demo
    instruction = f"保存代码到{file_name}\n#代码\n{code}" 
    edit_file_names=[{file_name}] # 要编辑的文件列表
    read_only_files=[] # 只读文件列表，只读文件不会被修改，是要编辑的文件依赖的文件
    result=aider_demo.aider_programming_demo.programming(instruction,edit_file_names,read_only_files) # 执行编程任务
    print(result) # 打印编程任务结果

    ## 如果指令是编写或修改或保存python文件，优先使用aider_demo.aider_programming_demo.programming函数执行编程任务修改python文件。如果programming函数失败，直接使用python代码修改文件。
                                ''')
    task_executer=Agent(llm=llm)
    
#%%
if __name__ == "__main__":
    response=coder.execute_stream('''
    写一个函数，计算斐波那契数列的第n个数.
    
    ''')
    result:Result=None
    for chunk in response:
        result=chunk
        print(chunk,end='',flush=True)
            
#%%
if __name__ == "__main__":
    response=code_file_editor.execute_stream(f'''
    把代码保存到/home/guci/myModule/AiResearch/hello_world.py

    # 代码
    {result.code}
                                                ''')
    result=None
    for chunk in response:
        result=chunk
        print(chunk,end='',flush=True)

#%%
if __name__ == "__main__":
    response=task_executer.execute_stream(f'''
    运行/home/guci/myModule/AiResearch/hello_world.py，判断测试是否通过
    ''')
    result=None
    for chunk in response:
        result=chunk
        print(chunk,end='',flush=True)
        
#%%
if __name__ == "__main__":
    print(result.return_value)    

# hello world

In [ ]:
import os

# os.environ['https_proxy'] = 'socks5://127.0.0.1:7890'
# os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'

# #%%
# # 取消代理服务器环境变量设置
# os.environ.pop('http_proxy', None)
# os.environ.pop('https_proxy', None) 
# os.environ.pop('all_proxy', None)



# 示例1：自主规划模式（默认）
from pythonTask import llm_llama_4_maverick_openrouter, llm_deepseek
llm = llm_deepseek

# 创建自主规划模式的多步骤智能体
multi_agent_autonomous = MultiStepAgent_v2(llm=llm, use_autonomous_planning=True)
coder_agent = Agent(llm=llm)
multi_agent_autonomous.register_agent(
    name="coder",
    instance=coder_agent
)

# 自主规划模式 - AI会自主分解任务
print("=== 自主规划模式示例 ===")
main_instruction = "请用python写一个hello world程序"
multi_agent_autonomous.execute_multi_step(main_instruction)


In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import json
load_dotenv()
llm_gemini_2_flash_openrouter = ChatOpenAI(
    temperature=0,
    model="google/gemini-2.0-flash-001", 
    base_url='https://openrouter.ai/api/v1',
    api_key=os.getenv('OPENROUTER_API_KEY')
)
prompt='''
请判断以下命题的真值，返回json格式，json只有一个字段，字段名是result，字段值是true或者false。

#命题
1+1=2
'''
response = llm_gemini_2_flash_openrouter.invoke(
    prompt, 
    response_format={"type": "json_object"}
)
x = response.content
print("Raw response:", x)

# 解析JSON响应
try:
    result = json.loads(x)
    print("Parsed JSON:", result)
    print("Result value:", result.get('result'))
except json.JSONDecodeError as e:
    print("JSON parsing error:", e)

# web research

In [ ]:

import os
# os.environ["AGENT_MAX_TOKENS"] = "1000000"
from pythonTask import *
from knowledge_agent import promgraming_knowledge
from aiTools import *

# llm=llm_gemini_2_5_pro_preview_05_06_google
llm=llm_deepseek


# 实例化 MultiStepAgent_v2 时不传入 agent_specs
multi_agent = MultiStepAgent_v2(llm=llm)


web_agent = Agent(llm=llm)
web_agent.loadPythonModules(['aiTools'])
web_agent.loadKnowledge('''
如果指令要求你写文档，你应该调用llm_gemini_2_flash_openrouter语言模型生成文档
## 调用语言模型示例
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
llm_gemini_2_flash_openrouter = ChatOpenAI(
temperature=0,
model="google/gemini-2.0-flash-001", 
base_url='https://openrouter.ai/api/v1',
api_key=os.getenv('OPENROUTER_API_KEY')
)
x:str=llm_gemini_2_flash_openrouter.invoke("你好").content
print(x)
''')
web_agent.api_specification="""
web_agent,擅长搜索网络信息,抓取网页内容，生成报告

# api 规范

## 生成报告
### 输入
1: 生成报告的指令，包含报告的要求和约束，这是必须的信息。
1：参考资料，比如收集到的网页内容，大小必须小于一百万token。这是可选的信息。
### 输出
完整的报告

# 示例

```python
with open('reference_document.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    
processed_content = content.strip()

prompt=f'''
请用中文写一份销售报告。

# 销售数据参考资料
{processed_content}
'''

response=web_agent.execute_stream(prompt)
result=None # response流最后一个元素是Result对象
for chunk in response:
    result=chunk
    print(chunk,end='',flush=True)

print('销售报告\n')
print(result.return_value)
```

"""

multi_agent.register_agent(
name="web_agent",
instance=web_agent
)


# 示例主指令
# main_instruction = "请用python写一个hello world程序"
main_instruction = """

# 任务
1:执行者web_agent,使用关键词crewai 搜索网络信息，抓取搜索结果的前5个网页，把抓取到的网页合并保存在web_agent智能体的web_content变量中
2:执行者web_agent,把web_agent智能体的web_content变量的内容传递给gemini模型生成crewai教程，教程必须使用中文。教程需包含crewai的系统架构，核心组件，示例代码。
3:教程保存在crewai_tutorial.md文件中

"""

# 执行多步骤任务
result = multi_agent.execute_multi_step(main_instruction)
print("多步骤执行结果：")
print(result)

# 总结

In [ ]:
if __name__ == "__main__":
    summary_prompt='''
    你的任务是创建一份详细的对话总结，密切关注用户的明确要求和你之前的行动。
    这份总结应该全面捕获技术细节、代码模式和架构决策，这些对于继续对话和支持任何后续任务都是必不可少的。
    你的总结应该按以下结构组织：
    上下文：继续对话所需的上下文。如果根据当前任务适用，这应该包括：
    之前的对话：关于整个与用户对话中讨论内容的高层次细节。这应该写得让别人能够跟上总体对话流程。
    当前工作：详细描述在此次总结请求之前正在进行的工作。特别注意对话中最近的消息。
    关键技术概念：列出所有重要的技术概念、技术、编码约定和讨论过的框架，这些可能与继续这项工作相关。
    相关文件和代码：如果适用，枚举为任务继续而检查、修改或创建的具体文件和代码部分。特别注意最近的消息和更改。
    问题解决：记录到目前为止解决的问题和任何正在进行的故障排除工作。
    待处理任务和下一步：概述你被明确要求处理的所有待处理任务，以及列出你将为所有未完成工作采取的下一步，如果适用的话。在能增加清晰度的地方包含代码片段。对于任何下一步，包含来自最近对话的直接引用，准确显示你正在处理的任务以及你停止的地方。这应该是逐字逐句的，以确保任务之间的上下文没有信息丢失。
    示例总结结构：
    之前的对话：
    [详细描述]
    当前工作：
    [详细描述]
    关键技术概念：
    [概念1]
    [概念2]
    [...]
    相关文件和代码：
    [文件名1]
    [此文件重要性的总结]
    [对此文件所做更改的总结，如有的话]
    [重要代码片段]
    [文件名2]
    [重要代码片段]
    [...]
    问题解决：
    [详细描述]
    待处理任务和下一步：
    [任务1详情和下一步]
    [任务2详情和下一步]
    [...]
    仅输出到目前为止的对话总结，不要添加任何其他评论或解释。
    '''

    resonse=multi_agent.chat_stream(summary_prompt)
    for chunk in resonse:
        print(chunk,end='',flush=True)

# 磁盘分析

In [ ]:
agent=Agent(llm=llm_deepseek)
response=agent.execute_stream('''
    打印出/home/guci目录下所有size大于500M的文件
                              ''')
for chunk in response:
    print(chunk,flush=True,end='')